In [1]:
import pandas as pd
import os, glob
import numpy as np
from sklearn.cross_validation import train_test_split
import tensorflow as tf

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('C:/Users/Admin/Desktop/CWD/DATA/MNIST REAL DATA/train.csv')
data.head()

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [3]:
import os.path
from skimage import io
cwd = 'C:/Users/Admin/Desktop/CWD/DATA/MNIST REAL DATA/Images/train'
X = data['filename']
Y = data['label']
print(X[5])
x = []
for image in X:
    x.append(os.path.join(cwd,image).replace("\\","/"))
x = np.array(x)


all_images = []
for image_path in x:
  img = io.imread(image_path , as_grey=True)
  img = img.reshape(img.shape[0]*img.shape[1],)
  all_images.append(img)
x_train = np.array(all_images)
y_train = np.array(Y)


5.png


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


onehot_encoder = OneHotEncoder(sparse=False)
y_train = y_train.reshape(len(y_train), 1)
y_train = onehot_encoder.fit_transform(y_train)
print(y_train)
print(x_train.shape)
print(y_train.shape)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
(49000, 784)
(49000, 10)


In [5]:
flat_image_size = x_train[1].shape[0]
no_of_classes = 10
n_h1 = 500
n_h2 = 450
n_h3 = 212
n_out = no_of_classes
x_train.shape
# y_train.shape

(49000, 784)

In [6]:
from sklearn.cross_validation import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size = 0.2)
type(x_test1)

numpy.ndarray

In [7]:
x = tf.placeholder('float', shape = [None, flat_image_size])
y = tf.placeholder('float', shape = [None, no_of_classes])

def neural_nets(data):
    hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([flat_image_size, n_h1])),
                      'bias': tf.Variable(tf.random_normal([n_h1]))}
    
    hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([n_h1, n_h2])),
                      'bias': tf.Variable(tf.random_normal([n_h2]))}
    
    hidden_layer_3 = {'weights': tf.Variable(tf.random_normal([n_h2, n_h3])),
                      'bias': tf.Variable(tf.random_normal([n_h3]))}
    
    output = {'weights': tf.Variable(tf.random_normal([n_h3, n_out])),
              'bias': tf.Variable(tf.random_normal([n_out]))}
    
    l1 = tf.add(tf.matmul(x,hidden_layer_1['weights']), hidden_layer_1['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_layer_2['weights']), hidden_layer_2['bias'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_layer_3['weights']), hidden_layer_3['bias'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output['weights']), output['bias'])
    output = tf.nn.relu(output)

    return output


In [ ]:
# def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
#     assert inputs.shape[0] == targets.shape[0]
#     if shuffle:
#         indices = np.arange(inputs.shape[0])
#         np.random.shuffle(indices)
#     for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
#         if shuffle:
#             excerpt = indices[start_idx:start_idx + batchsize]
#         else:
#             excerpt = slice(start_idx, start_idx + batchsize)
#         yield inputs[excerpt], targets[excerpt]

        
# for epoch in range(no_epoch):
#     epoch_loss = 0
#     for batch in iterate_minibatches(X, Y, batch_size, shuffle=True):
#         x_batch, y_batch = batch

In [16]:
def neural_nets_train(data):
    prediction = neural_nets(data)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    length_of_input_array = len(x_train)
    no_epoch = 300
    batch_size = 100
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(no_epoch):
            epoch_loss = 0
            print('------------------EPOCH:', epoch,'------------------')
            for j in range(0, length_of_input_array, batch_size):
                batch_train_X = x_train[j:j+batch_size]
                batch_train_y = y_train[j:j+batch_size]     

       
                o,c = sess.run([optimizer, cost], feed_dict = {x:batch_train_X, y:batch_train_y})
                epoch_loss +=c
                print('Epoch',epoch,'out of ',no_epoch,'Epochs','Loss: ',epoch_loss)
            
                correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
                accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
                
                feed_dict_train = {x: x_train1,
                                   y: y_train1}
                acc = sess.run(accuracy, feed_dict = feed_dict_train)
                print("Accuracy on train-set: {0:.1%}".format(acc))
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        feed_dict_test= {x: x_test1,
                         y: y_test1}
        
        acc = sess.run(accuracy, feed_dict = feed_dict_test)
        print("Accuracy on test-set: {0:.1%}".format(acc))
neural_nets_train(x_train)

------------------EPOCH: 0 ------------------
Epoch 0 out of  300 Epochs Loss:  29935.0683594
Accuracy on train-set: 13.2%
Epoch 0 out of  300 Epochs Loss:  56237.3125
Accuracy on train-set: 13.7%
Epoch 0 out of  300 Epochs Loss:  80682.5878906
Accuracy on train-set: 14.1%
Epoch 0 out of  300 Epochs Loss:  101168.080078
Accuracy on train-set: 14.8%
Epoch 0 out of  300 Epochs Loss:  122283.0
Accuracy on train-set: 15.3%
Epoch 0 out of  300 Epochs Loss:  140181.636719
Accuracy on train-set: 15.9%
Epoch 0 out of  300 Epochs Loss:  155858.787109
Accuracy on train-set: 16.5%
Epoch 0 out of  300 Epochs Loss:  172089.197266
Accuracy on train-set: 17.4%
Epoch 0 out of  300 Epochs Loss:  186111.709961
Accuracy on train-set: 18.7%
Epoch 0 out of  300 Epochs Loss:  198246.387695
Accuracy on train-set: 19.8%
Epoch 0 out of  300 Epochs Loss:  210294.525391
Accuracy on train-set: 21.0%
Epoch 0 out of  300 Epochs Loss:  221905.388672
Accuracy on train-set: 22.2%
Epoch 0 out of  300 Epochs Loss:  2321

Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341616.396296
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341618.65283
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341620.909364
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341623.188923
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341625.445457
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341627.725017
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341630.004576
Accuracy on train-set: 11.4%
Epoch 0 out of  300 Epochs Loss:  341632.238085
Accuracy on train-set: 11.3%
Epoch 0 out of  300 Epochs Loss:  341654.81677


KeyboardInterrupt: 